In [1]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os, sys

sys.path.append('../utilities')
from alignFaces import *

In [2]:
data_params = {
    "dataset":"../../facial_recognition_data/faces_dataset/", # path to input directory of faces and images
    "embeddings":"../../output/embeddings.pickle", # path to output of database of facial embeddings
    "detector":"../../facial_detection/models/", # path to OpenCV's deep learning face detector
    "embedding_model":"../models/openface.nn4.small2.v1.t7", # path to OpenCV's deep learning face embedding model
    "confidence":0.6 # min probability to filter weak detections
}

In [3]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join([data_params["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([data_params["detector"], "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(data_params["embedding_model"])

[INFO] loading face detector...
[INFO] loading face recognizer...


In [4]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images(data_params["dataset"]))
print(*imagePaths, sep="\n")
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

[INFO] quantifying faces...
../../facial_recognition_data/faces_dataset/andrew\far_frame0.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame112.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame126.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame14.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame140.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame154.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame168.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame182.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame196.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame210.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame224.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame238.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame252.jpg
../../facial_recognition_data/faces_dataset/andrew\far_frame266.jpg
../../facial_recognitio

In [5]:
# loop over the image paths
af = AlignFaces()
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
    # The right side of the last occurance of a '/' character is gotten from the path.
    # Then the string is split based on the '\' character which prepends the image name.
    # The left side of the split, representing the directory/person name is saved as name.
    name = imagePath[imagePath.rindex('/')+1:].split('\\')[0]
    image = cv2.imread(imagePath) # Load the image
    image = af.align_faces(image, imagePath) # Align the image
    if image is None:
        print("[INFO] no face found in image {}".format(i + 1))
        continue
    image = imutils.resize(image, width=600) # Resize the image (while maintaining aspect ratio)
    (h, w) = image.shape[:2] # Grab the image dimensions.
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    
    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]

        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > data_params["confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI and grab the ROI dimensions
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
            
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total += 1

[INFO] processing image 1/561
[INFO] processing image 2/561
[INFO] processing image 3/561
[INFO] processing image 4/561
[INFO] processing image 5/561
[INFO] processing image 6/561
[INFO] processing image 7/561
[INFO] processing image 8/561
[INFO] processing image 9/561
[INFO] processing image 10/561
[INFO] processing image 11/561
[INFO] processing image 12/561
[INFO] processing image 13/561
[INFO] processing image 14/561
[INFO] processing image 15/561
[INFO] processing image 16/561
[INFO] processing image 17/561
[INFO] processing image 18/561
[INFO] processing image 19/561
[INFO] processing image 20/561
[INFO] processing image 21/561
[INFO] processing image 22/561
[INFO] processing image 23/561
[INFO] processing image 24/561
[INFO] processing image 25/561
[INFO] processing image 26/561
[INFO] processing image 27/561
[INFO] processing image 28/561
[INFO] processing image 29/561
[INFO] processing image 30/561
[INFO] processing image 31/561
[INFO] processing image 32/561
[INFO] processing

[INFO] processing image 260/561
[INFO] processing image 261/561
[INFO] processing image 262/561
[INFO] processing image 263/561
[INFO] processing image 264/561
[INFO] processing image 265/561
[INFO] processing image 266/561
[INFO] processing image 267/561
[INFO] processing image 268/561
[INFO] processing image 269/561
[INFO] processing image 270/561
[INFO] processing image 271/561
[INFO] processing image 272/561
[INFO] processing image 273/561
[INFO] processing image 274/561
[INFO] processing image 275/561
[INFO] processing image 276/561
[INFO] processing image 277/561
[INFO] processing image 278/561
[INFO] processing image 279/561
[INFO] processing image 280/561
[INFO] processing image 281/561
[INFO] processing image 282/561
[INFO] processing image 283/561
[INFO] processing image 284/561
[INFO] processing image 285/561
[INFO] processing image 286/561
[INFO] processing image 287/561
[INFO] processing image 288/561
[INFO] processing image 289/561
[INFO] processing image 290/561
[INFO] p

[INFO] processing image 517/561
[INFO] processing image 518/561
[INFO] processing image 519/561
[INFO] processing image 520/561
[INFO] processing image 521/561
[INFO] processing image 522/561
[INFO] processing image 523/561
[INFO] processing image 524/561
[INFO] processing image 525/561
[INFO] processing image 526/561
[INFO] processing image 527/561
[INFO] processing image 528/561
[INFO] processing image 529/561
[INFO] processing image 530/561
[INFO] processing image 531/561
[INFO] processing image 532/561
[INFO] processing image 533/561
[INFO] processing image 534/561
[INFO] processing image 535/561
[INFO] processing image 536/561
[INFO] processing image 537/561
[INFO] processing image 538/561
[INFO] processing image 539/561
[INFO] processing image 540/561
[INFO] processing image 541/561
[INFO] processing image 542/561
[INFO] processing image 543/561
[INFO] processing image 544/561
[INFO] processing image 545/561
[INFO] processing image 546/561
[INFO] processing image 547/561
[INFO] p

In [6]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(data_params["embeddings"], "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 519 encodings...
